In [1]:
import ps3838api.api as ps

In [2]:
ps.get_client_balance()

{'availableBalance': 2242.47,
 'outstandingTransactions': 0.0,
 'givenCredit': 0.0,
 'currency': 'USD'}

In [3]:
import ps3838api.tank as tank

eventMatcher = tank.EventMatcher()

In [4]:
result = eventMatcher.get_odds('Italy Serie A', 'Lazio', 'Torino')
result

event founded
updating odds


{'id': 1605863576,
 'periods': [{'lineId': 3020273891,
   'number': 0,
   'cutoff': '2025-03-31T18:45:00Z',
   'spreadUpdatedAt': '2025-03-31T18:44:07.350Z',
   'moneylineUpdatedAt': '2025-03-31T18:44:07.350Z',
   'totalUpdatedAt': '2025-03-31T18:44:07.350Z',
   'teamTotalUpdatedAt': '2025-03-31T18:44:07.350Z',
   'status': 1},
  {'lineId': 3020389622,
   'number': 1,
   'cutoff': '2025-03-31T18:45:00Z',
   'spreadUpdatedAt': '2025-03-31T18:44:07.413Z',
   'moneylineUpdatedAt': '2025-03-31T18:44:07.413Z',
   'totalUpdatedAt': '2025-03-31T18:44:07.413Z',
   'teamTotalUpdatedAt': '2025-03-31T18:44:07.413Z',
   'status': 2}]}

In [6]:
result = eventMatcher.get_odds('Republic of Ireland First Division', 'UCD', 'Bray Wanderers')
result

NoSuchLeagueMatching(league='Republic of Ireland First Division')

In [7]:
result = eventMatcher.get_odds('France Ligue 2', 'Paris FC', 'Caen')
result

event founded
updating odds


{'id': 1606273674,
 'periods': [{'lineId': 3020276281,
   'number': 0,
   'cutoff': '2025-03-31T18:43:59Z',
   'spreadUpdatedAt': '2025-03-31T18:42:19.413Z',
   'moneylineUpdatedAt': '2025-03-31T18:42:19.413Z',
   'totalUpdatedAt': '2025-03-31T18:42:19.413Z',
   'teamTotalUpdatedAt': '2025-03-31T18:42:19.413Z',
   'status': 1},
  {'lineId': 3020386301,
   'number': 1,
   'cutoff': '2025-03-31T18:43:58Z',
   'spreadUpdatedAt': '2025-03-31T18:42:19.413Z',
   'moneylineUpdatedAt': '2025-03-31T18:42:19.413Z',
   'totalUpdatedAt': '2025-03-31T18:42:19.413Z',
   'teamTotalUpdatedAt': '2025-03-31T18:42:19.413Z',
   'status': 2}]}

In [ ]:
not_live_odds = ps.get_odds(sport_id=ps.SOCCER_SPORT_ID)
tank.filter_odds(not_live_odds, event_id=1606273674)

In [30]:
import json


with open('examples/live_odds.json', 'w') as file:
    json.dump(eventMatcher.odds.data, file, indent=4)

In [33]:
import json


with open('examples/fixtures.json', 'w') as file:
    json.dump(eventMatcher.fixtures.data, file, indent=4)

In [16]:
from typing import TypedDict
import json


class MatchedLeague(TypedDict):
    betsapi_league: str
    ps3838_league: str | None
    ps3838_id: int | None

In [17]:
with open('out/matched_leagues.json') as file:
    data: list[MatchedLeague] = json.load(file)

In [ ]:
england_championship = data[0]
england_championship

In [3]:
ps3838_leagues_id: list[int] = [x['ps3838_id'] for x in data if x['ps3838_id']]

In [19]:
import ps3838api.api as ps

response = ps.get_odds(ps.SOCCER_SPORT_ID, is_live=True) # type: ignore 

In [ ]:
response

In [8]:
with open('examples/get_odds.json', 'w') as file:
    json.dump(response, file, indent=4)

In [ ]:
from models.odds import OddsResponse

def find_over_bets(odds_data: OddsResponse) -> None:
    for league in odds_data["leagues"]:
        league_id = league["id"]
        for event in league["events"]:
            event_id = event["id"]
            for period in event["periods"]:
                # 'status' == 1 indicates "online" (open for betting)
                if period["status"] == 1:  
                    # 'totals' is a list of possible Over/Under lines
                    for total_line in period["totals"]:
                        # total_line is an OddsTotalV3
                        # total_line = cast(OddsTotalV3, total_line)
                        points = total_line["points"]    # e.g. 2.5
                        over_odds = total_line["over"]   # e.g. 1.87 (decimal)
                        under_odds = total_line["under"] # e.g. 1.95 (decimal)

                        # If you want to specifically look for an Over 2.5 line, for instance:
                        # if points == 2.5:
                        if "altLineId" in total_line:
                            line_id = total_line["altLineId"]
                        else:
                            line_id = ''
                        print(
                            f"League={league_id}, Event={event_id}, {line_id=}"
                            f"Period={period['number']} => Over {points} @ {over_odds} "
                            f"(Under @ {under_odds})"
                        )
                            # Then place bet with betType='TOTAL_POINTS', side='OVER', handicap=2.5, etc.


find_over_bets(response)

In [ ]:
ps.get_fixtures(ps.SOCCER_SPORT_ID, event_ids=[1606758261, ])